In [ ]:
%tensorflow_version 2.x
%load_ext tensorboard

import tensorflow as tf

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from os import path, walk
import numpy as np
import datetime

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
data_dir = path.join(path.curdir, "databases", "cajus-vermelhos")

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    image_size=(512, 512),
    batch_size=12,
    seed=123)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    image_size=(512, 512),
    batch_size=12,
    seed=123)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.shuffle(1000).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(512, 512, 3)))
model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.4, fill_mode="nearest"))
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="swish"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="swish"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="swish"))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="swish"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="swish"))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.SpatialDropout2D(0.5))

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(128, activation="elu"))
model.add(tf.keras.layers.Dense(64, activation="elu"))
model.add(tf.keras.layers.Dense(32, activation="elu"))
# model.add(tf.keras.layers.Dense(16, activation="swish"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1)

log_dir = "./logs/cajus-vermelhos/rgb/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [ ]:
history = model.fit(
    train_ds,
    epochs=1000,
    validation_data=val_ds,
    callbacks=[earlyStopping, tensorboard_callback]
)

In [ ]:
loss, acc = model.evaluate(val_ds)

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])

In [ ]:
! mkdir -p models/gabirobas

model.save('./models/cajus-vermelhos/rgb')

In [ ]:
%tensorboard --logdir ./logs/cajus-vermelhos/rgb/

In [ ]:
!tensorboard dev upload \
    --logdir ./logs/cajus-vermelhos/rgb/20210222-193928 \
    --name "cajus-vermelhos-rgb" \
    --description "cnn model on cajus-vermelhos rbg images" \
    --one_shot

In [ ]:
loaded_model = tf.keras.models.load_model('./models/cajus-vermelhos/rgb')

In [ ]:
loaded_model.to_json()